# Numeric Tests

This notebook runs some numerical tests to assess the validity, accuracy, and performance of numeric python packages for the Weierstrass elliptic functions. Where appropriate it compares two alternative versions. 

In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
# Original package https://github.com/stla/pyweierstrass/blob/main/pyweierstrass/weierstrass.py
from weierstrass import omega_from_g, g_from_omega, wp, invwp, wpprime, wsigma

# Modified package
from weierstrass_modified import Weierstrass
wst = Weierstrass()

from random import random
from mpmath import almosteq, mpc, mpf, im, timing, polyroots
from sympy import *

In [46]:
omega1, omega2, omega3, g2, g3, tau, G4, G6, q = symbols('omega1, omega2, omega3, g2, g3, tau, G4, G6, q')

## 1. Calculating the half-periods from the elliptic invariants

- The series for g2 in terms of G4 is quartic in omega1. When solving for omega1 in terms of g2 the result is thus only determined up to a quartic root of unity.
- If the series for g2 in terms of G4 is used the result for omega1 can be incorrectly multiplied by +1, -1, +i, or -i, we can observe errors when trying to recalculate g2 and g3 from omega and the mistake manifests as giving the wrong sign for g3.
- One solution is to include the series for g3 in terms of G6 to decide whether to multiply the periods by the imaginary unit or not, 
    then the periods are defined up to a sign +/-1. In other words, the ratio g2/g3 is quadratic in the half-periods not quartic. 
- As the half periods are only defined up to a sign because of periodicity anyway this suffices and elliminates the problem of obtaining the wrong sign for g3 when calculating from omega.
- By G4 and G6 we mean G4(1, tau), G6(1, tau) with G_2k(tau) = 2 * zeta(2*k) * E_2k(tau), with zeta the Riemann zeta function and E the Eisenstein series expressible in terms of theta functions
- The original package has since merged a fix for this issue and implemented the suggestion below. (see https://github.com/stla/pyweierstrass/issues/3) 
- The issue is resolved and the below tests are now passing.

So instead of doing this:

In [55]:
g2_G4_eq = Eq(g2, Rational(60, 2**4) * G4 / omega1**4)
g2_G4_eq

Eq(g2, 15*G4/(4*omega1**4))

In [56]:
Eq(omega1, (Rational(60, 2**4) * G4/ g2) ** (Rational(1, 4)))

Eq(omega1, 15**(1/4)*sqrt(2)*(G4/g2)**(1/4)/2)

It is better to do this:

In [62]:
g2_G4_eq

Eq(g2, 15*G4/(4*omega1**4))

In [57]:
g3_G6_eq = Eq(g3, Rational(140,2**6) * G6 / omega1**6 )
g3_G6_eq

Eq(g3, 35*G6/(16*omega1**6))

In [58]:
Eq(g2_G4_eq.lhs / g3_G6_eq.lhs, g2_G4_eq.rhs / g3_G6_eq.rhs)

Eq(g2/g3, 12*G4*omega1**2/(7*G6))

In [60]:
Eq(omega1, sqrt(Rational(7, 12) * g2 * G6 / (g3 * G4)))

Eq(omega1, sqrt(21)*sqrt(G6*g2/(G4*g3))/6)

Calulating half-periods from g2 and g3, then calculating g2 and g3 from the half-periods, then comparing the calculated values to the orignal g2 and g3 values.

In [78]:
def test_accuracy_of_g_from_omega_and_omega_from_g(
    omega_from_g_function,
    g_from_omega_function,
    Ntests = 1000,
    num_span = 100,
    tolerance = 1e-10
):

    err_count = 0
    for _ in range(Ntests):
        try:

            # Define random values for g2 and g3 
            # Picks values from within the complex square bounded by
            # -num_span ... num_span, -1i*num_span ... 1i*num_span
            g2_num = mpc(
                real=f'{num_span*2*(random() - 0.5)}', 
                imag=f'{num_span*2*(random() - 0.5)}'
            )
            g3_num = mpc(
                real=f'{num_span*2*(random() - 0.5)}', 
                imag=f'{num_span*2*(random() - 0.5)}'
            )

            # Calculate the half-periods from g2 and g3
            omegas = omega_from_g_function(g2_num, g3_num)
            omega1 = omegas[0]
            omega2 = omegas[1]

            # Swap the sign of omega2 if needed 
            # to get an allowed value for tau = omega2/omega1
            sign_flipped = False
            if im(omega2/omega1) <= 0:
                omega2 = -omega2
                sign_flipped = True

            # Calculate g2 and g3 from the half-periods
            g2calc, g3calc = g_from_omega_function(omega1, omega2)

            # Compare the accuracy of the calculated value of g2 to the original
            err_1 = False
            if not almosteq(g2_num, g2calc, tolerance):
                err_msg_1 = f'g2calc={g2calc} not within tolerance for g2_num={g2_num}'
                err_1 = True

            # Compare the accuracy of the calculated value of g3 to the original
            err_2 = False
            if not almosteq(g3_num, g3calc, tolerance):
                err_msg_2 = f'g3calc={g3calc} not within tolerance for g3_num={g3_num}'
                err_2 =True

            # Raise an exception if any calculated values for g2 and g3 
            # differ from the original values by more than the tolerance
            err_msg = ""
            if err_1:
                err_msg += err_msg_1 + '\n'
            if err_2:
                err_msg += err_msg_2 + '\n'
            if err_1 or err_2:
                raise Exception(err_msg)

        except Exception as e:
            
            # Track the error rate 
            err_count += 1

            if verbose:
                # Print the values and error messages 
                # for which accuracy was outside the tolerance
                print('Error:\n', e)
                print('g2',g2_num)
                print('g3',g3_num)
                print('sign_flipped', sign_flipped)

    # Calculate the error rate
    err_rate = err_count / Ntests
    print("Error rate:", err_rate)

In [79]:
# Using the original package this shows the problems that occur with incorrect half-periods
# leading to g3 taking the wrong sign approximately half the time
test_accuracy_of_g_from_omega_and_omega_from_g(
    omega_from_g,
    g_from_omega
)

0.0


In [80]:
# Using the modified package
test_accuracy_of_g_from_omega_and_omega_from_g(
    wst.omega_from_g,
    wst.g_from_omega
)

0.0


## 2. Calculating the roots e in terms of theta functions

- The roots e1, e2, e3 can be expressed in terms of jacobi theta functions: https://dlmf.nist.gov/23.6
- If the half-periods are known or already calculated, it is approximately 10x faster to calculate the roots e1, e2, e3 using known formulas in terms of theta functions than to numerically solve the cubic 4 * z**3 - g2 * z - g3 = 0
- Note: It seems the original package has now switched to using theta functions for the roots

In [80]:
g2_num = mpc(real=f'{num_span * 2*(random() - 0.5)}', imag=f'{num_span * 2*(random() - 0.5)}')
g3_num = mpc(real=f'{num_span * 2*(random() - 0.5)}', imag=f'{num_span * 2*(random() - 0.5)}')

# Calculate the half-periods from g2 and g3
omegas = wst.omega_from_g(g2_num, g3_num)
omega1 = omegas[0]
omega2 = omegas[1]

# Swap the sign of omega2 if needed to get an allowed value for tau = omega2/omega1
if im(omega2/omega1) <= 0:
    omega2 = -omega2

# Obtain the roots e1, e2, e3 using omega1 and omega2 with theta functions in the modified package    
e1_, e2_, e3_ = wst.roots_from_omega1_omega2(omega1, omega2)

# Obtain the roots e1, e2, e3 using g2 and g3, solve the cubic with the original package    
e1, e2, e3 = polyroots([4, 0, -g2_num, -g3_num])

print(e1_, e2_, e3_)
print(e1, e2, e3)

(4.21862608420373 - 0.541419183621149j) (-0.101791599340047 + 1.40040687435132j) (-4.11683448486369 - 0.858987690730171j)
(4.21862608420371 - 0.541419183621149j) (-4.11683448486366 - 0.858987690730169j) (-0.101791599340047 + 1.40040687435132j)


In [73]:
Ntests = 100
time_roots_from_omega = 0
time_roots_from_g = 0

for _ in range(Ntests):
    try:
        
        # Define random values for g2 and g3 
        # Picks values from within the complex square -num_span ... num_span, -1i*num_span ... 1i*num_span
        g2_num = mpc(real=f'{num_span * 2*(random() - 0.5)}', imag=f'{num_span * 2*(random() - 0.5)}')
        g3_num = mpc(real=f'{num_span * 2*(random() - 0.5)}', imag=f'{num_span * 2*(random() - 0.5)}')
        
        # Calculate the half-periods from g2 and g3
        omegas = wst.omega_from_g(g2_num, g3_num)
        omega1 = omegas[0]
        omega2 = omegas[1]

        # Swap the sign of omega2 if needed to get an allowed value for tau = omega2/omega1
        if im(omega2/omega1) <= 0:
            omega2 = -omega2
            
        # Obtain the roots e1, e2, e3 using omega1 and omega2 with theta functions and time it    
        time_roots_from_omega += timing(wst.roots_from_omega1_omega2, omega1, omega2)
        
        # Obtain the roots e1, e2, e3 using g2 and g3, solve the cubic and time it    
        time_roots_from_g += timing(wst.sorted_roots_from_g2_g3, g2_num, g3_num)
        
    except Exception as e:
        print(e)

# Calculate average times
average_time_roots_from_omega = time_roots_from_omega / Ntests
average_time_roots_from_g = time_roots_from_g / Ntests

print("average_time_roots_from_omega = ", average_time_roots_from_omega)
print("average_time_roots_from_g = ", average_time_roots_from_g)
print("average_time_roots_from_omega/average_time_roots_from_g = ", 
      average_time_roots_from_omega/average_time_roots_from_g)

average_time_roots_from_omega =  0.00024413072098104747
average_time_roots_from_g =  0.0015215324469863838
average_time_roots_from_omega/average_time_roots_from_g =  0.1604505519843391


## 3. Picking the correct option from a choice of two values when inverting the Weierstrass P function

- The Weierstrass Elliptic function is a meromorphic function of order 2 and thus there are two non-congruent solutions to the equation: wp(z) = c
- This means care must be taken to pick the right one when attempting to invert the Weierstrass function.
- If the value of the derivative, i.e. the value of Weirstrass P prime wpp at z, is known, this can be used to pick from the two possible choices of z.
- As the Weierstrass P function is even, if wp(z) = c then wp(-z) = c, but as Weierstrass P Prime is odd wpp(-z) = -wpp(z) hence if we know wpp(z) we can pick the right sign for z. 

## 4. Testing the accuracy of the inverse Weierstrass P function

In [61]:
def test_accuracy_of_inverse_weierstrass(
    omega_from_g_function,
    w_from_z_and_omega_function,
    inv_w_from_omega_and_w_function,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=False
):

    err_count = 0
    for _ in range(Ntests):
        try:

            # Define a random value for z
            # Picks values from within the complex bounded by 
            # -num_span ... num_span, -1i*num_span ... 1i*num_span
            z = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )

            # Define random values for g2 and g3 
            # Picks values from within the complex square bounded by 
            # -num_span ... num_span, -1i*num_span ... 1i*num_span
            g2_num = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )
            g3_num = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )

            # Calculate the half-periods from g2 and g3
            omegas = omega_from_g_function(g2_num, g3_num)
            omega2, omega1 = (omegas[0], omegas[1])

            # Swap the sign of omega2 if needed 
            # to get an allowed value for tau = omega2/omega1
            if im(omega2/omega1) <= 0:
                omega2 = -omega2
            omega = (omega1, omega2)

            # Calculate the Weierstrass P function at z
            w_num = w_from_z_and_omega_function(z, omega)

            # Calculate the value for z 
            # by evaluating the inverse Weierstrass P function 
            # at the corresponding value of Weierstrass P, w_num.
            # This should be equal to the starting value for z 
            # modulo lattice periods
            z_calc = inv_w_from_omega_and_w_function(w_num, omega)

            # Calculate the Weierstrass P function at z_calc
            # By comparing the values for Weierstrass P 
            # we can test the inversion function without dealing with
            # modulo lattice periods
            w_z_calc = w_from_z_and_omega_function(z_calc, omega)

            # Evaluate the difference between the Weierstrass P function
            # evaluated at the original z and the z obtained from the inverse Weierstrass
            if not almosteq(w_num, w_z_calc, tolerance):
                raise Exception(
                    '''
                    The value for Weierstrass P function evaluated at the original z value
                    and at the value of z calculated from the inverse Weierstrass function differ
                    by more than the tolerance.'
                    '''
                )

        except Exception as e:

            # Track the error count
            err_count += 1

            if verbose:
                print('Error:\n', e)
                print('w_num', w_num)
                print('w_z_calc', w_z_calc)
       
    # Calculate the error rate
    err_rate = err_count / Ntests
    print("Error rate:", err_rate)

In [25]:
# Original package
test_accuracy_of_inverse_weierstrass(
    omega_from_g,
    wp,
    invwp,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=True
)

Error rate: 0.0


In [26]:
# Modified package
test_accuracy_of_inverse_weierstrass(
    wst.omega_from_g,
    wst.wp,
    wst.invwp,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=True
)

Error rate: 0.0


# 5. Testing the accuracy of Weierstrass P Prime (Weierstrass P derivative) 

In [62]:
def test_accuracy_of_weierstrass_p_prime(
    omega_from_g_function,
    w_from_z_and_omega_function,
    w_prime_from_z_and_omega_function,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=False
):
    """
    Note: this test does not check the sign of Weierstrass P Prime as its value is squared
    """

    err_count = 0
    for _ in range(Ntests):
        try:

            # Define a random value for z
            # Picks values from within the complex square bounded by 
            # -num_span ... num_span, -1i*num_span ... 1i*num_span
            z = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )

            # Define random values for g2 and g3 
            # Picks values from within the complex bounded by 
            # -num_span ... num_span, -1i*num_span ... 1i*num_span
            g2_num = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )
            g3_num = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )

            # Calculate the half-periods from g2 and g3
            omegas = omega_from_g_function(g2_num, g3_num)
            omega2, omega1 = (omegas[0], omegas[1])

            # Swap the sign of omega2 if needed 
            # to get an allowed value for tau = omega2/omega1
            if im(omega2/omega1) <= 0:
                omega2 = -omega2
            omega = (omega1, omega2)

            # Calculate the Weierstrass P function at z
            # and the value of the corresponding cubic
            w_num = w_from_z_and_omega_function(z, omega)
            w_prime_cubic = 4 * w_num ** 3 - g2_num * w_num - g3_num

            # Calculate the Weierstrass P Prime function at z and its square
            w_prime = w_prime_from_z_and_omega_function(z, omega)
            w_prime_sqrd = w_prime ** 2

            # Evaluate the difference between the Weierstrass P function in the cubic
            # and the Weierstrass P Prime function squared
            if not almosteq(w_prime_cubic, w_prime_sqrd, tolerance):
                raise Exception(
                    '''
                    The value for Weierstrass P function evaluated in the cubic
                    and the value for Weierstrass P Prime squared differ
                    by more than the tolerance.'
                    '''
                )

        except Exception as e:

            # Track the error count
            err_count += 1

            if verbose:
                print('Error:\n', e)
                print('w_prime_cubic', w_prime_cubic)
                print('w_prime_sqrd', w_prime_sqrd)
       
    # Calculate the error rate
    err_rate = err_count / Ntests
    print("Error rate:", err_rate)

In [81]:
# Original package
test_accuracy_of_weierstrass_p_prime(
    omega_from_g,
    wp,
    wpprime,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=True
)

Error rate: 0.0


In [58]:
# Modified package
test_accuracy_of_weierstrass_p_prime(
    wst.omega_from_g,
    wst.wp,
    wst.wpprime,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=True
)

Error rate: 0.0


## 6. Testing the accuracy of Weierstrass Sigma

In [75]:
def test_accuracy_of_weierstrass_sigma(
    omega_from_g_function,
    w_from_z_and_omega_function,
    sigma_from_z_and_omega_function,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=False
):
    
    """
    This tests identity 23.10.3 here https://dlmf.nist.gov/23.10
    which relates Weierstrass Sigma function to Weierstrass P
    """

    err_count = 0
    for _ in range(Ntests):
        try:

            # Define two random values for u and v
            # Picks values from within the complex square bounded by 
            # -num_span ... num_span, -1i*num_span ... 1i*num_span
            u = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )
            v = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )

            # Define random values for g2 and g3 
            # Picks values from within the complex bounded by 
            # -num_span ... num_span, -1i*num_span ... 1i*num_span
            g2_num = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )
            g3_num = mpc(
                real=f'{num_span * 2*(random() - 0.5)}', 
                imag=f'{num_span * 2*(random() - 0.5)}'
            )

            # Calculate the half-periods from g2 and g3
            omegas = omega_from_g_function(g2_num, g3_num)
            omega2, omega1 = (omegas[0], omegas[1])

            # Swap the sign of omega2 if needed 
            # to get an allowed value for tau = omega2/omega1
            if im(omega2/omega1) <= 0:
                omega2 = -omega2
            omega = (omega1, omega2)

            # Calculate the Weierstrass P function at u and v
            # then calculate the Weierstrass P side of the identity
            wp_num_u = w_from_z_and_omega_function(u, omega)
            wp_num_v = w_from_z_and_omega_function(v, omega)
            wp_side_of_eq = wp_num_v - wp_num_u

            # Calculate the Weierstrass Sigma function at 
            # u + v, u - v, and its square at u and v
            # then calculate the sigma side of the identity
            sigma_u_plus_v = sigma_from_z_and_omega_function(u + v, omega)
            sigma_u_minus_v = sigma_from_z_and_omega_function(u - v, omega)
            sigma_u_sqrd = sigma_from_z_and_omega_function(u, omega) ** 2
            sigma_v_sqrd = sigma_from_z_and_omega_function(v, omega) ** 2
            sigma_side_of_eq = sigma_u_plus_v * sigma_u_minus_v / sigma_u_sqrd / sigma_v_sqrd

            # Evaluate the difference between the Weierstrass P side of the identity
            # and the Weierstrass Sigma side of the identity
            if not almosteq(wp_side_of_eq, sigma_side_of_eq, tolerance):
                raise Exception(
                    '''
                    The value for the Weierstrass P side of the identity
                    and the value for Weierstrass Sigma side of the identity
                    differ by more than the tolerance.'
                    '''
                )

        except Exception as e:

            # Track the error count
            err_count += 1

            if verbose:
                print('Error:\n', e)
                print('wp_side_of_eq', wp_side_of_eq)
                print('sigma_side_of_eq', sigma_side_of_eq)
       
    # Calculate the error rate
    err_rate = err_count / Ntests
    print("Error rate:", err_rate)

In [82]:
# Original package
test_accuracy_of_weierstrass_sigma(
    omega_from_g,
    wp,
    wsigma,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=True
)

Error rate: 0.0


In [77]:
# Modified package
test_accuracy_of_weierstrass_sigma(
    wst.omega_from_g,
    wst.wp,
    wst.wsigma,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=True
)

Error rate: 0.0


## 7. Testing the accuracy of Weierstrass Zeta

In [ ]:
def test_accuracy_of_weierstrass_zeta(
    omega_from_g_function,
    w_from_z_and_omega_function,
    zeta_from_z_and_omega_function,
    Ntests=1000,
    num_span=100,
    tolerance=1e-10,
    verbose=False
):

    err_count = 0
    for _ in range(Ntests):
        try:

            ## TO DO

        except Exception as e:

            # Track the error count
            err_count += 1

            if verbose:
                print('Error:\n', e)
       
    # Calculate the error rate
    err_rate = err_count / Ntests
    print("Error rate:", err_rate)